In [1]:
import os
import tensorflow as tf
if tf.__version__!='2.11.0':
  !pip install tensorflow==2.11.0 --quiet
  os.kill(os.getpid(), 9)

In [2]:
!pip install keras-rl2 --quiet
!pip install gym[atari] --quiet
!pip install atari-py --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.1/52.1 kB 672.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.6/540.6 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!python -m atari_py.import_roms /content/drive/MyDrive/dqn/roms/

copying breakout.bin from /content/drive/MyDrive/dqn/roms/Breakout - Breakaway IV.bin to /usr/local/lib/python3.10/dist-packages/atari_py/atari_roms/breakout.bin


In [5]:
import gym
from gym.envs.registration import register

/usr/local/lib/python3.10/dist-packages/ale_py/roms/__init__.py:89: DeprecationWarning: Automatic importing of atari-py roms won't be supported in future releases of ale-py. Please migrate over to using `ale-import-roms` OR an ALE-supported ROM package. To make this warning disappear you can run `ale-import-roms --import-from-pkg atari_py.atari_roms`.For more information see: https://github.com/mgbellemare/Arcade-Learning-Environment#rom-management
  ROMS = resolve_roms()


In [6]:
register(
    id='Breakout-v4',
    entry_point='gym.envs.atari:AtariEnv',
    kwargs={'game': 'breakout', 'obs_type': 'image', 'frameskip': 1},
    max_episode_steps=10000,
    nondeterministic=False,
)

/usr/local/lib/python3.10/dist-packages/gym/envs/registration.py:542: UserWarning: WARN: Overriding environment Breakout-v4
  logger.warn(f"Overriding environment {spec.id}")


In [18]:
import numpy as np
import gym

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Convolution2D, Permute
from tensorflow.keras.optimizers.legacy import Adam
from keras.callbacks import Callback

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint

In [19]:
ENV_NAME = 'Breakout-v4'

# Get Environment.
env = gym.make(ENV_NAME)
np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n

# Testing to simplify the model
model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))

policy = EpsGreedyQPolicy()
memory = SequentialMemory(limit=1000000, window_length=1)
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=10000,
               target_model_update=1e-2, policy=policy)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

# Define the callback
class CustomCallback(Callback):
    def on_step_end(self, step, logs={}):
        if step % 100 == 0:
            print('Average reward at step {}: {}'.format(step, logs.get('episode_reward')))

# Training and saving
weights_filename = f'/content/drive/MyDrive/dqn/weights.h5'

dqn.fit(env, nb_steps=100000, visualize=False, verbose=2, callbacks=[CustomCallback()])
dqn.save_weights(weights_filename, overwrite=True)

/usr/local/lib/python3.10/dist-packages/gym/envs/atari/environment.py:68: UserWarning: WARN: obs_type "image" should be replaced with the image type, one of: rgb, grayscale
  logger.warn(


Training for 100000 steps ...
Average reward at step 0: None


/usr/local/lib/python3.10/dist-packages/keras/engine/training_v1.py:2357: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:227: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  logger.deprecation(


Average reward at step 100: None
Average reward at step 200: None
Average reward at step 300: None
Average reward at step 400: None
Average reward at step 500: None
Average reward at step 600: None
Average reward at step 700: None
Average reward at step 800: None
Average reward at step 900: None
Average reward at step 1000: None
  1056/100000: episode: 1, duration: 9.127s, episode steps: 1056, steps per second: 116, episode reward:  2.000, mean reward:  0.002 [ 0.000,  1.000], mean action: 1.951 [0.000, 3.000],  loss: --, mae: --, mean_q: --
Average reward at step 0: None
Average reward at step 100: None
Average reward at step 200: None
Average reward at step 300: None
Average reward at step 400: None
Average reward at step 500: None
Average reward at step 600: None
Average reward at step 700: None
Average reward at step 800: None
Average reward at step 900: None
Average reward at step 1000: None
Average reward at step 1100: None
  2216/100000: episode: 2, duration: 7.232s, episode ste

/usr/local/lib/python3.10/dist-packages/keras/engine/training_v1.py:2357: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


Average reward at step 200: None
Average reward at step 300: None
Average reward at step 400: None
Average reward at step 500: None
Average reward at step 600: None
Average reward at step 700: None
Average reward at step 800: None
 10694/100000: episode: 10, duration: 276.110s, episode steps: 852, steps per second:   3, episode reward:  2.000, mean reward:  0.002 [ 0.000,  1.000], mean action: 1.231 [0.000, 3.000],  loss: 2879.371607, mae: 3.594421, mean_q: 4.867226
Average reward at step 0: None
done, took 365.483 seconds


In [21]:
dqn.test(env, nb_episodes=10, visualize=False)

Testing for 10 episodes ...
Episode 1: reward: 0.000, steps: 487
Episode 2: reward: 0.000, steps: 486
Episode 3: reward: 0.000, steps: 486
Episode 4: reward: 0.000, steps: 486
Episode 5: reward: 0.000, steps: 486
Episode 6: reward: 0.000, steps: 486
Episode 7: reward: 0.000, steps: 489
Episode 8: reward: 0.000, steps: 486
Episode 9: reward: 0.000, steps: 487
Episode 10: reward: 0.000, steps: 486


In [27]:
weights_filename = f'/content/drive/MyDrive/dqn/weights.h5'
dqn.load_weights(weights_filename)
dqn.test(env, nb_episodes=10, visualize=False)

Testing for 10 episodes ...
Episode 1: reward: 0.000, steps: 10000
Episode 2: reward: 0.000, steps: 10000
Episode 3: reward: 0.000, steps: 10000
Episode 4: reward: 0.000, steps: 10000
Episode 5: reward: 0.000, steps: 10000
Episode 6: reward: 0.000, steps: 10000
Episode 7: reward: 0.000, steps: 10000
Episode 8: reward: 0.000, steps: 10000
Episode 9: reward: 0.000, steps: 10000
Episode 10: reward: 0.000, steps: 10000
